In [1]:
# Script demonstrating pruning of a Fully Connected layer with bias.

In [2]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import pdb
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                       transform=transform)

train_kwargs = {'batch_size': 64}
test_kwargs = {'batch_size': 1000}
if torch.cuda.is_available():
    cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

In [4]:
def train(log_interval, model, device, train_loader, optimizer, epoch, alpha =1):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data, alpha)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {}, alpha: {}, [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, alpha, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))



def test(model, device, test_loader, alpha=1):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data, alpha)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Alpha {}, Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(alpha,
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [5]:
# Original Pre-trained model (Trained using mnist.py)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 1, 3, 1, bias = False)
        self.conv2 = nn.Conv2d(1, 2, 3, 1, bias = False)
        self.conv3 = nn.Conv2d(2, 256, 3, 1, bias = False)
        
        self.bn1 = nn.BatchNorm2d(1)
        self.bn2 = nn.BatchNorm2d(2)
        self.bn3 = nn.BatchNorm2d(256)
        #self.dropout1 = nn.Dropout(0.25)
        #self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(256 * 1 * 1, 128)
        self.fc2 = nn.Linear(128, 50)
        self.fc3 = nn.Linear(50, 10)
        

    def forward(self, x, alpha = 1):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        #x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        #x = self.dropout2(x)
        x = self.fc2(x)
        x= F.relu(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output
    
model = Net().to(device)

In [6]:
# Load weights
PATH = 'mnist_cnn.pt'
model_dict= torch.load(PATH, map_location = device)
model.load_state_dict(model_dict)

<All keys matched successfully>

In [7]:
# Test to see if the weights of the pretrained model have been correctly loaded
test(model, device, test_loader)


Test set: Alpha 1, Average loss: 0.1157, Accuracy: 9643/10000 (96%)



In [8]:
# A pretrinaed Network with a branch parrallel to FC1 for the purpose of redcucing from 128 to 64 neurons
# See also image 'highlevel.png'. However, note that the FC layers also have a bias
class PrunedNet(nn.Module):
    def __init__(self,pretrainedmodel):
        super(PrunedNet, self).__init__()
        self.pretrainedNet = pretrainedmodel
        
        # No need to update weights of the pretrained network        
        for param in self.pretrainedNet.parameters():
            param.requires_grad=False
        
        # In case the weights of the pretrained network are also desired to be updated then 
        # comment out the above 2 lines of code
        
        # Branch parallel to the fc1 layer in the original pre-trained network
        self.fcc = nn.Linear(128, 64)
        self.fcd = nn.Linear(64, 128)
        
    def forward(self, x, alpha=0.1):
        x = self.pretrainedNet.conv1(x)
        x = self.pretrainedNet.bn1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.pretrainedNet.conv2(x)
        x = self.pretrainedNet.bn2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.pretrainedNet.conv3(x)
        x = self.pretrainedNet.bn3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        xorg = torch.flatten(x, 1)
        x1 = self.pretrainedNet.fc1(xorg)
        
        x2 = self.fcd((F.relu(self.fcc(x1))))

        x = alpha * F.relu(x1) + (1-alpha) * x2

        
        x = self.pretrainedNet.fc2(x)

        x= F.relu(x)
        
        x = self.pretrainedNet.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output
pruned_model = PrunedNet(model).to(device)

In [25]:
# Check to see if the original branch(alpha = 1 ) gives the same accuracy.
# If not, then something wrong with the implementation
print('Original accuracy:')
test(model, device, test_loader)
print('Accuracy of pruned_model at alpha = 1:')
print(test(pruned_model, device, test_loader, alpha = 1))

Original accuracy:

Test set: Alpha 1, Average loss: 0.1156, Accuracy: 9641/10000 (96%)

Accuracy of pruned_model at alpha = 1:

Test set: Alpha 1, Average loss: 0.1156, Accuracy: 9641/10000 (96%)

None


In [9]:
# 2 For loops, The outer is for varying the alpha values. The inner loop are the training epochs
# The schedule of how alpha is decayed is something to explore.
# In this case, a simple, linear decay has been used.
optimizer = optim.Adadelta(filter(lambda p: p.requires_grad, pruned_model.parameters()), lr=1.0)
for alpha in np.arange(0.9,-0.01,-0.1):
    scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
    for epoch in range(1, 10 + 1):
        train(10,pruned_model, device, train_loader, optimizer, epoch, alpha)
        test(pruned_model, device, test_loader, alpha)
        scheduler.step()

Train Epoch: 1, alpha: 0.9, [0/60000 (0%)]	Loss: 0.064865
Train Epoch: 1, alpha: 0.9, [640/60000 (1%)]	Loss: 0.109263
Train Epoch: 1, alpha: 0.9, [1280/60000 (2%)]	Loss: 0.063188
Train Epoch: 1, alpha: 0.9, [1920/60000 (3%)]	Loss: 0.193504
Train Epoch: 1, alpha: 0.9, [2560/60000 (4%)]	Loss: 0.096558
Train Epoch: 1, alpha: 0.9, [3200/60000 (5%)]	Loss: 0.047871
Train Epoch: 1, alpha: 0.9, [3840/60000 (6%)]	Loss: 0.078465
Train Epoch: 1, alpha: 0.9, [4480/60000 (7%)]	Loss: 0.056746
Train Epoch: 1, alpha: 0.9, [5120/60000 (9%)]	Loss: 0.138127
Train Epoch: 1, alpha: 0.9, [5760/60000 (10%)]	Loss: 0.017473
Train Epoch: 1, alpha: 0.9, [6400/60000 (11%)]	Loss: 0.071419
Train Epoch: 1, alpha: 0.9, [7040/60000 (12%)]	Loss: 0.160465
Train Epoch: 1, alpha: 0.9, [7680/60000 (13%)]	Loss: 0.083776
Train Epoch: 1, alpha: 0.9, [8320/60000 (14%)]	Loss: 0.060530
Train Epoch: 1, alpha: 0.9, [8960/60000 (15%)]	Loss: 0.124979
Train Epoch: 1, alpha: 0.9, [9600/60000 (16%)]	Loss: 0.086981
Train Epoch: 1, alpha

Train Epoch: 2, alpha: 0.9, [23040/60000 (38%)]	Loss: 0.065421
Train Epoch: 2, alpha: 0.9, [23680/60000 (39%)]	Loss: 0.062165
Train Epoch: 2, alpha: 0.9, [24320/60000 (41%)]	Loss: 0.042590
Train Epoch: 2, alpha: 0.9, [24960/60000 (42%)]	Loss: 0.034333
Train Epoch: 2, alpha: 0.9, [25600/60000 (43%)]	Loss: 0.048730
Train Epoch: 2, alpha: 0.9, [26240/60000 (44%)]	Loss: 0.130488
Train Epoch: 2, alpha: 0.9, [26880/60000 (45%)]	Loss: 0.091798
Train Epoch: 2, alpha: 0.9, [27520/60000 (46%)]	Loss: 0.135115
Train Epoch: 2, alpha: 0.9, [28160/60000 (47%)]	Loss: 0.156749
Train Epoch: 2, alpha: 0.9, [28800/60000 (48%)]	Loss: 0.025178
Train Epoch: 2, alpha: 0.9, [29440/60000 (49%)]	Loss: 0.090416
Train Epoch: 2, alpha: 0.9, [30080/60000 (50%)]	Loss: 0.041759
Train Epoch: 2, alpha: 0.9, [30720/60000 (51%)]	Loss: 0.072167
Train Epoch: 2, alpha: 0.9, [31360/60000 (52%)]	Loss: 0.041731
Train Epoch: 2, alpha: 0.9, [32000/60000 (53%)]	Loss: 0.084879
Train Epoch: 2, alpha: 0.9, [32640/60000 (54%)]	Loss: 0

Train Epoch: 3, alpha: 0.9, [46720/60000 (78%)]	Loss: 0.100861
Train Epoch: 3, alpha: 0.9, [47360/60000 (79%)]	Loss: 0.149965
Train Epoch: 3, alpha: 0.9, [48000/60000 (80%)]	Loss: 0.087177
Train Epoch: 3, alpha: 0.9, [48640/60000 (81%)]	Loss: 0.048400
Train Epoch: 3, alpha: 0.9, [49280/60000 (82%)]	Loss: 0.091378
Train Epoch: 3, alpha: 0.9, [49920/60000 (83%)]	Loss: 0.061143
Train Epoch: 3, alpha: 0.9, [50560/60000 (84%)]	Loss: 0.041772
Train Epoch: 3, alpha: 0.9, [51200/60000 (85%)]	Loss: 0.032976
Train Epoch: 3, alpha: 0.9, [51840/60000 (86%)]	Loss: 0.181745
Train Epoch: 3, alpha: 0.9, [52480/60000 (87%)]	Loss: 0.044775
Train Epoch: 3, alpha: 0.9, [53120/60000 (88%)]	Loss: 0.037777
Train Epoch: 3, alpha: 0.9, [53760/60000 (90%)]	Loss: 0.015643
Train Epoch: 3, alpha: 0.9, [54400/60000 (91%)]	Loss: 0.011969
Train Epoch: 3, alpha: 0.9, [55040/60000 (92%)]	Loss: 0.132210
Train Epoch: 3, alpha: 0.9, [55680/60000 (93%)]	Loss: 0.080050
Train Epoch: 3, alpha: 0.9, [56320/60000 (94%)]	Loss: 0

Train Epoch: 5, alpha: 0.9, [8960/60000 (15%)]	Loss: 0.098959
Train Epoch: 5, alpha: 0.9, [9600/60000 (16%)]	Loss: 0.040151
Train Epoch: 5, alpha: 0.9, [10240/60000 (17%)]	Loss: 0.031487
Train Epoch: 5, alpha: 0.9, [10880/60000 (18%)]	Loss: 0.031226
Train Epoch: 5, alpha: 0.9, [11520/60000 (19%)]	Loss: 0.076600
Train Epoch: 5, alpha: 0.9, [12160/60000 (20%)]	Loss: 0.052836
Train Epoch: 5, alpha: 0.9, [12800/60000 (21%)]	Loss: 0.070610
Train Epoch: 5, alpha: 0.9, [13440/60000 (22%)]	Loss: 0.035442
Train Epoch: 5, alpha: 0.9, [14080/60000 (23%)]	Loss: 0.065365
Train Epoch: 5, alpha: 0.9, [14720/60000 (25%)]	Loss: 0.224036
Train Epoch: 5, alpha: 0.9, [15360/60000 (26%)]	Loss: 0.102312
Train Epoch: 5, alpha: 0.9, [16000/60000 (27%)]	Loss: 0.044273
Train Epoch: 5, alpha: 0.9, [16640/60000 (28%)]	Loss: 0.062880
Train Epoch: 5, alpha: 0.9, [17280/60000 (29%)]	Loss: 0.086526
Train Epoch: 5, alpha: 0.9, [17920/60000 (30%)]	Loss: 0.097544
Train Epoch: 5, alpha: 0.9, [18560/60000 (31%)]	Loss: 0.1

Train Epoch: 6, alpha: 0.9, [32000/60000 (53%)]	Loss: 0.103381
Train Epoch: 6, alpha: 0.9, [32640/60000 (54%)]	Loss: 0.022332
Train Epoch: 6, alpha: 0.9, [33280/60000 (55%)]	Loss: 0.019405
Train Epoch: 6, alpha: 0.9, [33920/60000 (57%)]	Loss: 0.056742
Train Epoch: 6, alpha: 0.9, [34560/60000 (58%)]	Loss: 0.033892
Train Epoch: 6, alpha: 0.9, [35200/60000 (59%)]	Loss: 0.058756
Train Epoch: 6, alpha: 0.9, [35840/60000 (60%)]	Loss: 0.074616
Train Epoch: 6, alpha: 0.9, [36480/60000 (61%)]	Loss: 0.155390
Train Epoch: 6, alpha: 0.9, [37120/60000 (62%)]	Loss: 0.112970
Train Epoch: 6, alpha: 0.9, [37760/60000 (63%)]	Loss: 0.063607
Train Epoch: 6, alpha: 0.9, [38400/60000 (64%)]	Loss: 0.010166
Train Epoch: 6, alpha: 0.9, [39040/60000 (65%)]	Loss: 0.090379
Train Epoch: 6, alpha: 0.9, [39680/60000 (66%)]	Loss: 0.080977
Train Epoch: 6, alpha: 0.9, [40320/60000 (67%)]	Loss: 0.092957
Train Epoch: 6, alpha: 0.9, [40960/60000 (68%)]	Loss: 0.052026
Train Epoch: 6, alpha: 0.9, [41600/60000 (69%)]	Loss: 0

...

Train Epoch: 5, alpha: 2.220446049250313e-16, [59520/60000 (99%)]	Loss: 0.045931

Test set: Alpha 2.220446049250313e-16, Average loss: 0.1585, Accuracy: 9622/10000 (96%)

Train Epoch: 6, alpha: 2.220446049250313e-16, [0/60000 (0%)]	Loss: 0.115345
Train Epoch: 6, alpha: 2.220446049250313e-16, [640/60000 (1%)]	Loss: 0.039439
Train Epoch: 6, alpha: 2.220446049250313e-16, [1280/60000 (2%)]	Loss: 0.082591
Train Epoch: 6, alpha: 2.220446049250313e-16, [1920/60000 (3%)]	Loss: 0.041508
Train Epoch: 6, alpha: 2.220446049250313e-16, [2560/60000 (4%)]	Loss: 0.017313
Train Epoch: 6, alpha: 2.220446049250313e-16, [3200/60000 (5%)]	Loss: 0.098086
Train Epoch: 6, alpha: 2.220446049250313e-16, [3840/60000 (6%)]	Loss: 0.006661
Train Epoch: 6, alpha: 2.220446049250313e-16, [4480/60000 (7%)]	Loss: 0.083619
Train Epoch: 6, alpha: 2.220446049250313e-16, [5120/60000 (9%)]	Loss: 0.012279
Train Epoch: 6, alpha: 2.220446049250313e-16, [5760/60000 (10%)]	Loss: 0.059774
Train Epoch: 6, alpha: 2.220446049250313e-

Train Epoch: 7, alpha: 2.220446049250313e-16, [3840/60000 (6%)]	Loss: 0.014643
Train Epoch: 7, alpha: 2.220446049250313e-16, [4480/60000 (7%)]	Loss: 0.008245
Train Epoch: 7, alpha: 2.220446049250313e-16, [5120/60000 (9%)]	Loss: 0.090843
Train Epoch: 7, alpha: 2.220446049250313e-16, [5760/60000 (10%)]	Loss: 0.129605
Train Epoch: 7, alpha: 2.220446049250313e-16, [6400/60000 (11%)]	Loss: 0.204780
Train Epoch: 7, alpha: 2.220446049250313e-16, [7040/60000 (12%)]	Loss: 0.035681
Train Epoch: 7, alpha: 2.220446049250313e-16, [7680/60000 (13%)]	Loss: 0.044773
Train Epoch: 7, alpha: 2.220446049250313e-16, [8320/60000 (14%)]	Loss: 0.018049
Train Epoch: 7, alpha: 2.220446049250313e-16, [8960/60000 (15%)]	Loss: 0.256608
Train Epoch: 7, alpha: 2.220446049250313e-16, [9600/60000 (16%)]	Loss: 0.008387
Train Epoch: 7, alpha: 2.220446049250313e-16, [10240/60000 (17%)]	Loss: 0.022835
Train Epoch: 7, alpha: 2.220446049250313e-16, [10880/60000 (18%)]	Loss: 0.082370
Train Epoch: 7, alpha: 2.220446049250313e

Train Epoch: 8, alpha: 2.220446049250313e-16, [8960/60000 (15%)]	Loss: 0.061448
Train Epoch: 8, alpha: 2.220446049250313e-16, [9600/60000 (16%)]	Loss: 0.035318
Train Epoch: 8, alpha: 2.220446049250313e-16, [10240/60000 (17%)]	Loss: 0.082603
Train Epoch: 8, alpha: 2.220446049250313e-16, [10880/60000 (18%)]	Loss: 0.156437
Train Epoch: 8, alpha: 2.220446049250313e-16, [11520/60000 (19%)]	Loss: 0.012246
Train Epoch: 8, alpha: 2.220446049250313e-16, [12160/60000 (20%)]	Loss: 0.096735
Train Epoch: 8, alpha: 2.220446049250313e-16, [12800/60000 (21%)]	Loss: 0.040048
Train Epoch: 8, alpha: 2.220446049250313e-16, [13440/60000 (22%)]	Loss: 0.059334
Train Epoch: 8, alpha: 2.220446049250313e-16, [14080/60000 (23%)]	Loss: 0.001179
Train Epoch: 8, alpha: 2.220446049250313e-16, [14720/60000 (25%)]	Loss: 0.015946
Train Epoch: 8, alpha: 2.220446049250313e-16, [15360/60000 (26%)]	Loss: 0.044004
Train Epoch: 8, alpha: 2.220446049250313e-16, [16000/60000 (27%)]	Loss: 0.037389
Train Epoch: 8, alpha: 2.22044

Train Epoch: 9, alpha: 2.220446049250313e-16, [14080/60000 (23%)]	Loss: 0.031292
Train Epoch: 9, alpha: 2.220446049250313e-16, [14720/60000 (25%)]	Loss: 0.091693
Train Epoch: 9, alpha: 2.220446049250313e-16, [15360/60000 (26%)]	Loss: 0.040574
Train Epoch: 9, alpha: 2.220446049250313e-16, [16000/60000 (27%)]	Loss: 0.099491
Train Epoch: 9, alpha: 2.220446049250313e-16, [16640/60000 (28%)]	Loss: 0.050335
Train Epoch: 9, alpha: 2.220446049250313e-16, [17280/60000 (29%)]	Loss: 0.135254
Train Epoch: 9, alpha: 2.220446049250313e-16, [17920/60000 (30%)]	Loss: 0.035665
Train Epoch: 9, alpha: 2.220446049250313e-16, [18560/60000 (31%)]	Loss: 0.051074
Train Epoch: 9, alpha: 2.220446049250313e-16, [19200/60000 (32%)]	Loss: 0.071718
Train Epoch: 9, alpha: 2.220446049250313e-16, [19840/60000 (33%)]	Loss: 0.069327
Train Epoch: 9, alpha: 2.220446049250313e-16, [20480/60000 (34%)]	Loss: 0.212112
Train Epoch: 9, alpha: 2.220446049250313e-16, [21120/60000 (35%)]	Loss: 0.029372
Train Epoch: 9, alpha: 2.220

Train Epoch: 10, alpha: 2.220446049250313e-16, [19200/60000 (32%)]	Loss: 0.016822
Train Epoch: 10, alpha: 2.220446049250313e-16, [19840/60000 (33%)]	Loss: 0.034762
Train Epoch: 10, alpha: 2.220446049250313e-16, [20480/60000 (34%)]	Loss: 0.038946
Train Epoch: 10, alpha: 2.220446049250313e-16, [21120/60000 (35%)]	Loss: 0.046668
Train Epoch: 10, alpha: 2.220446049250313e-16, [21760/60000 (36%)]	Loss: 0.073072
Train Epoch: 10, alpha: 2.220446049250313e-16, [22400/60000 (37%)]	Loss: 0.074884
Train Epoch: 10, alpha: 2.220446049250313e-16, [23040/60000 (38%)]	Loss: 0.013839
Train Epoch: 10, alpha: 2.220446049250313e-16, [23680/60000 (39%)]	Loss: 0.098054
Train Epoch: 10, alpha: 2.220446049250313e-16, [24320/60000 (41%)]	Loss: 0.118953
Train Epoch: 10, alpha: 2.220446049250313e-16, [24960/60000 (42%)]	Loss: 0.110022
Train Epoch: 10, alpha: 2.220446049250313e-16, [25600/60000 (43%)]	Loss: 0.043833
Train Epoch: 10, alpha: 2.220446049250313e-16, [26240/60000 (44%)]	Loss: 0.033701
Train Epoch: 10,

In [14]:
# Test to see the performance of the parallel branch by setting alpha = 0
# If accuracy is still high(approx. equal to when alpha =1),
# then it hints that the parrallel branch by itself is enough.
# Hence, the original pretrained model can then be pruned to a smaller size
print(test(pruned_model, device, test_loader, 0 ))
print(test(pruned_model, device, test_loader, 1 ))


Test set: Alpha 0, Average loss: 0.1583, Accuracy: 9620/10000 (96%)

None

Test set: Alpha 1, Average loss: 0.1156, Accuracy: 9641/10000 (96%)

None


In [21]:
# Final pruned model after doing the appropriate matrix(or tensor) multiplications
# Note that FC1 output has now only 64 neurons instead of 128
# Likewise, the input of FC2 has 64 instead of 128 neurons.
# The bias terms for FC1 and FC2 are also appropriately updated
class SmallerNet(nn.Module):
    def __init__(self):
        super(SmallerNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 1, 3, 1, bias = False)
        self.conv2 = nn.Conv2d(1, 2, 3, 1, bias = False)
        self.conv3 = nn.Conv2d(2, 256, 3, 1, bias = False)
        
        self.bn1 = nn.BatchNorm2d(1)
        self.bn2 = nn.BatchNorm2d(2)
        self.bn3 = nn.BatchNorm2d(256)
        #self.dropout1 = nn.Dropout(0.25)
        #self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(256 * 1 * 1, 64)
        self.fc2 = nn.Linear(64, 50)
        self.fc3 = nn.Linear(50, 10)
        

    def forward(self, x, alpha = 1):
        # Note that alpha is not being used here
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        #x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        #x = self.dropout2(x)
        x = self.fc2(x)
        x= F.relu(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output
    
smaller_model = SmallerNet().to(device)

In [22]:
# Since the model is randomly initilaized, the accuracy is low
print(test(smaller_model, device, test_loader, 0 ))


Test set: Alpha 0, Average loss: 2.3050, Accuracy: 1028/10000 (10%)

None


In [23]:
# The pretrained weights are loaded after the appropriate matrix multiplications to get the new weight matrix 
# and new bias terms

from copy import deepcopy
smaller_model_dict= deepcopy(model.state_dict())
print(smaller_model_dict["fc1.weight"].shape)
print(smaller_model_dict["fc2.weight"].shape)
smaller_model_dict["fc1.weight"] = pruned_model.state_dict()["fcc.weight"] @ pruned_model.state_dict()["pretrainedNet.fc1.weight"]
smaller_model_dict["fc2.weight"] = pruned_model.state_dict()["pretrainedNet.fc2.weight"] @ pruned_model.state_dict()["fcd.weight"] 
print(smaller_model_dict["fc1.weight"].shape)
print(smaller_model_dict["fc2.weight"].shape)

print(smaller_model_dict["fc1.bias"].shape)
print(smaller_model_dict["fc2.bias"].shape)

smaller_model_dict["fc1.bias"]= (pruned_model.state_dict()["fcc.weight"] @ pruned_model.state_dict()["pretrainedNet.fc1.bias"].unsqueeze(1)).squeeze() + pruned_model.state_dict()["fcc.bias"]
smaller_model_dict["fc2.bias"] = (pruned_model.state_dict()["pretrainedNet.fc2.weight"] @ pruned_model.state_dict()["fcd.bias"].unsqueeze(1)).squeeze()  +  pruned_model.state_dict()["pretrainedNet.fc2.bias"]

print(smaller_model_dict["fc1.bias"].shape)
print(smaller_model_dict["fc2.bias"].shape)

#smaller_model_dict.keys()
smaller_model.load_state_dict(smaller_model_dict)

torch.Size([128, 256])
torch.Size([50, 128])
torch.Size([64, 256])
torch.Size([50, 64])
torch.Size([128])
torch.Size([50])
torch.Size([64])
torch.Size([50])


<All keys matched successfully>

In [24]:
# Test accuracy of the smaller_model is the same as pruned_model when alpha=0
# Note that only pruned_model needs alpha values not the smaller_model(Check the forward functions for both)
# However, for the sake of simplicity, test() and train() funcitons were defined assuming that
# alpha is needed. But this alpha when passed to the smaller_model is irrelevant.
# TODO: Reimplement the train() and test() functions such that alpha is only required for pruned_model
print(test(smaller_model, device, test_loader, 0 ))



Test set: Alpha 0, Average loss: 0.1583, Accuracy: 9620/10000 (96%)

None


In [25]:
pruned_model.state_dict().keys()

odict_keys(['pretrainedNet.conv1.weight', 'pretrainedNet.conv2.weight', 'pretrainedNet.conv3.weight', 'pretrainedNet.bn1.weight', 'pretrainedNet.bn1.bias', 'pretrainedNet.bn1.running_mean', 'pretrainedNet.bn1.running_var', 'pretrainedNet.bn1.num_batches_tracked', 'pretrainedNet.bn2.weight', 'pretrainedNet.bn2.bias', 'pretrainedNet.bn2.running_mean', 'pretrainedNet.bn2.running_var', 'pretrainedNet.bn2.num_batches_tracked', 'pretrainedNet.bn3.weight', 'pretrainedNet.bn3.bias', 'pretrainedNet.bn3.running_mean', 'pretrainedNet.bn3.running_var', 'pretrainedNet.bn3.num_batches_tracked', 'pretrainedNet.fc1.weight', 'pretrainedNet.fc1.bias', 'pretrainedNet.fc2.weight', 'pretrainedNet.fc2.bias', 'pretrainedNet.fc3.weight', 'pretrainedNet.fc3.bias', 'fcc.weight', 'fcd.weight'])

In [106]:
test(smaller_model, device, test_loader, 0 )


Test set: Alpha 0, Average loss: 0.1413, Accuracy: 9634/10000 (96%)



In [102]:
print(smaller_model.state_dict()["fc1.weight"].shape)
print(smaller_model.state_dict()["fc2.weight"].shape)
print("--"*10)
print(model.state_dict()["fc1.weight"].shape)
print(model.state_dict()["fc2.weight"].shape)
print("--"*10)
print(pruned_model.state_dict()["pretrainedNet.fc1.weight"].shape)
print(pruned_model.state_dict()["fcc.weight"].shape)
print((pruned_model.state_dict()["fcc.weight"] @ pruned_model.state_dict()["pretrainedNet.fc1.weight"]).shape)
print("--"*10)
print(pruned_model.state_dict()["pretrainedNet.fc2.weight"].shape)
print(pruned_model.state_dict()["fcd.weight"].shape)

torch.Size([64, 256])
torch.Size([50, 64])
--------------------
torch.Size([128, 256])
torch.Size([50, 128])
--------------------
torch.Size([128, 256])
torch.Size([64, 128])
torch.Size([64, 256])
--------------------
torch.Size([50, 128])
torch.Size([128, 64])


In [71]:
pruned_model.state_dict()["fcc.weight"].shape
pruned_model.state_dict()["pretrainedNet.fc1.weight"]

torch.Size([64, 128])

In [32]:
class PrunedNet(nn.Module):
    def __init__(self,pretrainedmodel):
        super(PrunedNet, self).__init__()
        self.pretrainedNet = pretrainedmodel
        
        # No need to update weights of the pretrained network        
        for param in self.pretrainedNet.parameters():
            param.requires_grad=False
        
        self.fca = nn.Linear(256, 30, bias = False)
        self.fcb = nn.Linear(30, 128, bias = False)
        
        self.fcc = nn.Linear(128, 10, bias = False)
        self.fcd = nn.Linear(10, 50, bias = False)
        
    def forward(self, x, alpha=0.9):
        x = self.pretrainedNet.conv1(x)
        x = self.pretrainedNet.bn1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.pretrainedNet.conv2(x)
        x = self.pretrainedNet.bn2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.pretrainedNet.conv3(x)
        x = self.pretrainedNet.bn3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        xorg = torch.flatten(x, 1)
        x1 = self.pretrainedNet.fc1(xorg)
        x2 = self.fca(xorg)
        x2 = self.fcb(x2)
        #x = alpha * x1 + (1-alpha) * x2
        xorg = F.relu(x2)
        
        #x1 = self.pretrainedNet.fc2(xorg)
        x2 = self.fcc(xorg)
        x2 = self.fcd(x2)
        #x = alpha * x1 + (1-alpha) * x2
        x= F.relu(x2)
        
        x = self.pretrainedNet.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output
pruned_model = PrunedNet(model).to(device)

8

In [26]:
for p in pruned_model.pretrainedNet.parameters():
    pass
p.requires_grad

True

In [ ]:
class smallerNet(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 1, 3, 1, bias = False)
        self.conv2 = nn.Conv2d(1, 2, 3, 1, bias = False)
        self.conv3 = nn.Conv2d(2, 256, 3, 1, bias = False)
        
        self.bn1 = nn.BatchNorm2d(1)
        self.bn2 = nn.BatchNorm2d(2)
        self.bn3 = nn.BatchNorm2d(256)
        #self.dropout1 = nn.Dropout(0.25)
        #self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(256 * 1 * 1, 64, bias =False)
        self.fc2 = nn.Linear(64, 50, bias=False)
        self.fc3 = nn.Linear(50, 10, bias = False)
        

    def forward(self, x, alpha = 1):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        #x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        #x = self.dropout2(x)
        x = self.fc2(x)
        x= F.relu(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output
    
smaller_model = smallerNet().to(device)

In [11]:
layer3 = model_dict['conv3.weight'].cpu().numpy()
num_filters = layer3.shape[0]

In [12]:
layer3.shape

(256, 2, 3, 3)

In [13]:
# class PrunedNet(nn.Module):
#     def __init__(self,pretrainedmodel):
#         super(PrunedNet, self).__init__()
#         self.pretrainedNet = pretrainedmodel
        
#         # No need to update weights of the pretrained network        
#         for param in self.pretrainedNet.parameters():
#             param.requires_grad=False
        
#         self.fca = nn.Linear(256, 30, bias = False)
#         self.fcb = nn.Linear(30, 128, bias = False)
        
#         self.fcc = nn.Linear(128, 10, bias = False)
#         self.fcd = nn.Linear(10, 50, bias = False)
        
#     def forward(self, x, alpha=0.9):
#         x = self.pretrainedNet.conv1(x)
#         x = self.pretrainedNet.bn1(x)
#         x = F.relu(x)
#         x = F.max_pool2d(x, 2)
#         x = self.pretrainedNet.conv2(x)
#         x = self.pretrainedNet.bn2(x)
#         x = F.relu(x)
#         x = F.max_pool2d(x, 2)
#         x = self.pretrainedNet.conv3(x)
#         x = self.pretrainedNet.bn3(x)
#         x = F.relu(x)
#         x = F.max_pool2d(x, 2)
        
#         xorg = torch.flatten(x, 1)
#         x1 = self.pretrainedNet.fc1(xorg)
#         x2 = self.fca(xorg)
#         x2 = self.fcb(x2)
#         x = alpha * x1 + (1-alpha) * x2
#         xorg = F.relu(x)
        
#         x1 = self.pretrainedNet.fc2(xorg)
#         x2 = self.fcc(xorg)
#         x2 = self.fcd(x2)
#         x = alpha * x1 + (1-alpha) * x2
#         x= F.relu(x)
        
#         x = self.pretrainedNet.fc3(x)
#         output = F.log_softmax(x, dim=1)
#         return output
# pruned_model = PrunedNet(model).to(device)

256

In [14]:
scores = np.sum(np.sum(np.sum(np.abs(layer3),3),2),1)
print(scores)

[2.8458521 2.4906092 2.182644  2.4704623 2.604546  2.7074933 2.4273744
 2.4395137 2.7046404 2.2236407 2.724819  2.346136  2.4736838 2.704204
 2.352558  1.8051548 2.0248778 2.1634579 2.0994892 2.1173074 2.4371142
 2.56626   2.1074808 2.8850985 2.6320398 2.3986905 2.6123188 2.4185576
 2.3762336 2.3464718 2.3099728 2.3896155 2.3797746 2.571484  2.682601
 2.457549  2.5476065 2.703555  2.171604  2.8341658 2.7164273 2.3056388
 2.2262316 2.4162507 2.162299  2.1605859 2.3533275 2.154541  2.29719
 2.3968291 2.1048923 2.3813574 2.5073893 2.5824394 2.5691903 2.173055
 2.6351976 2.549543  2.6614966 2.5639572 2.6166143 2.8284776 2.1105638
 2.4044895 2.4763665 2.1381226 2.629759  2.30014   2.2500648 2.2014756
 1.8681186 1.7940662 2.735925  2.3465033 2.6203418 2.2923179 2.656416
 2.6224637 2.6273053 2.1582756 2.4212275 1.8324049 2.1633458 2.0751681
 2.4310641 2.3555272 2.2806036 2.3307126 2.0461018 2.3769479 2.0755684
 2.1711507 2.357153  2.3072407 2.2286    2.5354738 2.273097  2.405572
 2.4387927 2.

In [15]:
ordered_indices = np.argsort(scores)
ordered_indices

array([222,  71, 124,  15,  81, 179,  70, 229, 173, 123, 217, 119, 180,
       129, 161, 238, 213,  16, 206, 187, 105,  88, 139, 197, 209,  83,
        90, 131, 214, 177,  18, 166,  50, 143,  22,  62, 137, 127, 210,
        19, 133,  65, 171, 220, 208, 245, 144, 184,  47, 215,  79,  45,
        44,  82,  17,  91,  38, 150,  55, 128,   2, 120, 151, 191,  69,
       130, 159, 252, 181,   9,  42,  94, 227, 196, 231, 236, 247,  68,
        99, 138, 182, 116, 218,  96, 167,  86, 234, 235,  75, 233,  48,
        67,  41,  93,  30, 100, 107, 110, 135, 189,  87, 140, 154, 163,
        11,  29,  73, 205, 242,  14,  46, 125, 237,  85, 115,  92, 240,
       230, 228, 103, 142, 108,  28,  89, 145,  32,  51, 147, 255, 165,
       226, 212,  31,  49,  25, 117, 188,  63,  97, 168, 169, 104, 241,
       194,  43,  27,  80, 244,   6,  84, 109,  20, 232,  98,   7, 162,
       134, 198,  35, 126, 155, 132,   3, 114,  12,  64, 192, 246, 136,
       219, 221, 249,   1, 195,  52, 160, 172, 253, 122, 248, 20

In [16]:
num_filters_to_retain = int(num_filters * (1 - remove_ratio))
filters_idxs_to_retain = ordered_indices[:num_filters_to_retain]

In [17]:
model_dict['conv3.weight'][filters_idxs_to_retain].shape

torch.Size([192, 2, 3, 3])

In [18]:
model_dict['conv3.weight'] = model_dict['conv3.weight'][filters_idxs_to_retain]
model_dict["bn3.weight"] = model_dict['bn3.weight'][filters_idxs_to_retain]
model_dict["bn3.running_mean"] = model_dict['bn3.running_mean'][filters_idxs_to_retain]
model_dict["bn3.running_var"] = model_dict['bn3.running_var'][filters_idxs_to_retain]
model_dict["bn3.bias"] = model_dict['bn3.bias'][filters_idxs_to_retain]


In [19]:
model_dict["fc1.weight"] = model_dict["fc1.weight"][:,filters_idxs_to_retain]

In [20]:
#print(model_dict["fc1.weight"].shape)
#print(model_dict["fc1.weight"][:,filters_idxs_to_retain].shape)


In [26]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 1, 3, 1, bias = False)
        self.conv2 = nn.Conv2d(1, 2, 3, 1, bias = False)
        self.conv3 = nn.Conv2d(2, 192, 3, 1, bias = False)
        
        self.bn1 = nn.BatchNorm2d(1)
        self.bn2 = nn.BatchNorm2d(2)
        self.bn3 = nn.BatchNorm2d(192)
        #self.dropout1 = nn.Dropout(0.25)
        #self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(192 * 1 * 1, 128)
        self.fc2 = nn.Linear(128, 50)
        self.fc3 = nn.Linear(50, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        #x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        #x = self.dropout2(x)
        x = self.fc2(x)
        x= F.relu(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output
    
model = Net().to(device)

In [27]:
model.load_state_dict(model_dict)


<All keys matched successfully>

In [28]:
test(model, device, test_loader)


Test set: Average loss: 0.3935, Accuracy: 8718/10000 (87%)



In [29]:
optimizer = optim.Adadelta(model.parameters(), lr=1.0)

scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(1, 10 + 1):
    train(10,model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.204891
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.123856
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.205159
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.117286
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.038898
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.289284
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.196593
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.238233
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.399758
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.115703
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.118389
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.515954
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.176568
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.185592
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.064054
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.101278
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.185965
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.115036
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.062826
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.106442
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.078401
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.059897
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.054985
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.357186
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.062702
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.161579
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.172899
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.123033
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.111696
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.147828
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.193112
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.046242
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.065079
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.226090
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.143976
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.107194
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.069249
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.118811
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.031427
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.037398


Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.004859
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.296020
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.071183
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.029004
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.087584
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.138139
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.175175
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.055548
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.197740
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.010896
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.051935
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.009342
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.217856
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.067858
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.081472
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.093157
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.087819
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.129756
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.073688
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.134740


Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.082824
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.102972
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.052483
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.074387
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.156844
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.004268
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.046857
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.047494
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.017852
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.074836
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.013387
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.055583
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.110941
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.008111
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.010436
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.075385
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.070993
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.130795
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.121007
Train Epoch: 6 [25600/60000 (43%)]	Loss: 0.018806


Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.006007
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.056902

Test set: Average loss: 0.1118, Accuracy: 9671/10000 (97%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.092088
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.093092
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.055554
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.061812
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.085422
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.003094
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.127153
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.020580
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.156190
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.026704
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.036243
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.025266
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.057215
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.018690
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.043181
Train Epoch: 8 [9600/60000 (16%)]	Loss: 0.058149
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.059936
Train Epoch: 8 [10

Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.024038
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.203296
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.006788
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.131806
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.034274
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.014773
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.015448
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.049023
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.003238
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.023971
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.032843
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.009613
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.024686
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.032912
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.025528
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.370933
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.131681
Train Epoch: 9 [54400/60000 (91%)]	Loss: 0.043389
Train Epoch: 9 [55040/60000 (92%)]	Loss: 0.066766
Train Epoch: 9 [55680/60000 (93%)]	Loss: 0.117187


In [23]:
torch.save(model, "pruned_model.pth")

/usr/wiss/khamuham/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/wiss/khamuham/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/wiss/khamuham/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BatchNorm2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/wiss/khamuham/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " 

In [24]:
new_model = torch.load("pruned_model.pth")

In [25]:
test(new_model, device, test_loader)


Test set: Average loss: 0.1149, Accuracy: 9673/10000 (97%)



In [26]:
import numpy as np
import torch
from torch.autograd import Variable

x = Variable(torch.Tensor([5]),requires_grad=True)
w0 = Variable(torch.Tensor([2]),requires_grad=False)

w1 = Variable(torch.Tensor([3]),requires_grad=False)


y1 = x**2*w0
y2 = y1*w1


gradients = []
y1.register_hook(lambda x: gradients.append(x))
y2.register_hook(lambda x: gradients.append(x))
x.register_hook(lambda x: gradients.append(x))

y2.backward()

gradients

[tensor([1.]), tensor([3.]), tensor([60.])]

In [27]:
def print_value(value):
    print("The value is {}".format(value.item()))

x = Variable(torch.Tensor([5]),requires_grad=True)
w0 = Variable(torch.Tensor([2]),requires_grad=False)

w1 = Variable(torch.Tensor([3]),requires_grad=False)


y1 = x**2*w0
y2 = y1*w1


gradients = []
y1.register_hook(lambda x: print_value(x))
y2.register_hook(lambda x: print_value(x))
x.register_hook(lambda x: print_value(x))



In [28]:
y2.backward()

The value is 1.0
The value is 3.0
The value is 60.0
